In [1]:
from CRYSTALpytools.crystal_io import *
from CRYSTALpytools.convert import *
import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import *
from phonopy import Phonopy
from phonopy.structure.atoms import PhonopyAtoms
from os.path import join
import shutil

import os
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core.periodic_table import *

from mendeleev import element

'''import sys
sys.path.insert(1,'../../../Imperial/crystal-code-tools/CRYSTALpy/CRYSTALpy/')

from crystal_io import *
from convert import *
import os
import shutil as sh'''

"import sys\nsys.path.insert(1,'../../../Imperial/crystal-code-tools/CRYSTALpy/CRYSTALpy/')\n\nfrom crystal_io import *\nfrom convert import *\nimport os\nimport shutil as sh"

In [65]:
def get_delta_angle(structure, range_angle, conv_matrix=[]):
    # Return a list of delta angles from a range of %

    import numpy as np
    from pymatgen.core.structure import Structure

    if len(conv_matrix) == 0:
        conv_matrix = np.identity(3)

    lattice = np.matmul(conv_matrix,structure.lattice.matrix)

    angle = Structure(lattice,[],[]).lattice.alpha

    delta_angle = []
    for i in np.arange(range_angle[0],range_angle[1]+range_angle[2]/2,range_angle[2]):
        delta_angle.append(angle*i)

    return np.round(delta_angle,8)

def get_delta_lattice(structure, range_lattice, conv_matrix=[]):
    # Return a list of delta lattice from a range of %

    import numpy as np
    from pymatgen.core.structure import Structure

    if len(conv_matrix) == 0:
        conv_matrix = np.identity(3)

    lattice = np.matmul(conv_matrix,structure.lattice.matrix)

    lattice = Structure(lattice,[],[]).lattice.a

    delta_lattice = []
    for i in np.arange(range_lattice[0],range_lattice[1]+range_lattice[2]/2,range_lattice[2]):
        delta_lattice.append(lattice*i)

    return np.round(delta_lattice,8)

def set_angle(structure, delta_angle, conv_matrix=[]):
    # structure: primitive cell structure
    # delta_angle: delta on the conventional cell angle
    # conv_matrix: conversion matrix from the primitive to conventional cell
    
    import numpy as np
    from pymatgen.core.structure import Structure

    to_rad = np.pi/180
    to_deg = 180/np.pi


    if len(conv_matrix) == 0:
        conv_matrix = np.identity(3)

    conv_matrix_inv = np.linalg.inv(conv_matrix)
    lattice = np.matmul(conv_matrix,structure.lattice.matrix)


    A = lattice[0,:]
    B = lattice[1,:]
    C = lattice[2,:]

    len_A = np.linalg.norm(lattice[0,:])
    len_B = np.linalg.norm(lattice[1,:])
    len_C = np.linalg.norm(lattice[2,:])

    A_norm = A/len_A
    B_norm = B/len_B
    C_norm = C/len_C

    alpha = np.arccos(np.dot(B_norm,C_norm))*(180/np.pi)
    beta = np.arccos(np.dot(A_norm,C_norm))*(180/np.pi)
    gamma = np.arccos(np.dot(A_norm,B_norm))*(180/np.pi)

    vector_sum = A_norm + B_norm + C_norm
    len_vector_sum = np.linalg.norm(vector_sum)

    vector_sum_norm = vector_sum/len_vector_sum

    D_a = np.cross(np.cross(A_norm,vector_sum_norm),A_norm)
    D_b = np.cross(np.cross(B_norm,vector_sum_norm),B_norm)
    D_c = np.cross(np.cross(C_norm,vector_sum_norm),C_norm)

    len_D_a = np.linalg.norm(D_a)
    len_D_b = np.linalg.norm(D_b)
    len_D_c = np.linalg.norm(D_c)

    D_a_norm = D_a/len_D_a
    D_b_norm = D_b/len_D_b
    D_c_norm = D_c/len_D_c

    alpha_t = np.arccos(np.dot(A_norm,vector_sum_norm))*(180/np.pi)
    beta_t = np.arccos(np.dot(B_norm,vector_sum_norm))*(180/np.pi)
    gamma_t = np.arccos(np.dot(C_norm,vector_sum_norm))*(180/np.pi)

    len_vector_sum_p = np.sqrt(3 + 2*(np.cos((alpha+delta_angle)*to_rad) +
                                        np.cos((beta+delta_angle)*to_rad) +
                                        np.cos((gamma+delta_angle)*to_rad)))


    alpha_p = (np.arccos((  1+ np.cos((beta+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
    beta_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
    gamma_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((beta+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg


    delta_alpha_t = (alpha_t-alpha_p)*to_rad
    delta_beta_t = (beta_t-beta_p)*to_rad
    delta_gamma_t = (gamma_t-gamma_p)*to_rad

    #print(alpha,alpha_t,alpha_p,delta_alpha_t)

    A1 = (np.cos(delta_alpha_t)*A_norm + np.sin(delta_alpha_t)*D_a_norm) #* len_A
    B1 = (np.cos(delta_beta_t)*B_norm + np.sin(delta_beta_t)*D_b_norm) #* len_B
    C1 = (np.cos(delta_gamma_t)*C_norm + np.sin(delta_gamma_t)*D_c_norm) #* len_C

    new_lattice_conv = np.array([A1,B1,C1])

    new_lattice_conv[0,:] = new_lattice_conv[0,:] *(len_A)
    new_lattice_conv[1,:] = new_lattice_conv[1,:] *(len_B)
    new_lattice_conv[2,:] = new_lattice_conv[2,:] *(len_C)

    new_lattice_prim = np.matmul(conv_matrix_inv,new_lattice_conv)


    return Structure(new_lattice_prim,structure.atomic_numbers,structure.frac_coords)  

def set_cell(structure, delta_cell, conv_matrix=[]):
    # structure: primitive cell structure
    # delta_cell: delta on the conventional cell vectors
    # conv_matrix: conversion matrix from the primitive to conventional cell
    
    import numpy as np
    from pymatgen.core.structure import Structure

    if len(conv_matrix) == 0:
        conv_matrix = np.identity(3)

    conv_matrix_inv = np.linalg.inv(conv_matrix)
    lattice = np.matmul(conv_matrix,structure.lattice.matrix)


    A = lattice[0,:]
    B = lattice[1,:]
    C = lattice[2,:]

    len_A = np.linalg.norm(lattice[0,:])
    len_B = np.linalg.norm(lattice[1,:])
    len_C = np.linalg.norm(lattice[2,:])


    ###
    lattice_new = np.zeros((3,3))
    lattice_new[0,:] = (lattice[0,:]/len_A) * (len_A+delta_cell)
    lattice_new[1,:] = (lattice[1,:]/len_B) * (len_B+delta_cell)
    lattice_new[2,:] = (lattice[2,:]/len_C) * (len_C+delta_cell)
    
    prim_lattice_new = np.matmul(conv_matrix_inv,lattice_new)
    
    return Structure(prim_lattice_new,structure.atomic_numbers,structure.frac_coords)

def spiral(X, Y):
    x = y = 0
    dx = 0
    dy = -1
    x_list = []
    y_list = []
    for i in range(max(X, Y)**2):
        if (-X/2 < x <= X/2) and (-Y/2 < y <= Y/2):
            #print (x, y)
            x_list.append(x)
            y_list.append(y)
        if x == y or (x < 0 and x == -y) or (x > 0 and x == 1-y):
            dx, dy = -dy, dx
        x, y = x+dx, y+dy
    return x_list, y_list 

# First things first

In [78]:
# Initial structure

# From gui 
structure_prim_cry = Crystal_gui().read_cry_gui('./data/MnO.gui')
structure_prim_pmg = cry_gui2pmg(structure_prim_cry)
mno_pmg_opt = cry_gui2pmg(mno_gui_opt)

# From pymatgen
#

# Read CRYSTAL input file
prim_crystal_input = Crystal_input().from_file('./data/MnO.d12') #No GUESSP...
scel_crystal_input = Crystal_input().from_file('./data/MnO_supercell.d12') #No GUESSP...

#Other settings
range_lattice = [-0.01,0.01,0.005]
range_angle = [-0.01,0.01,0.005]
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]]) # If working with the primitive

# slurm file
slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
slurm_data = file.readlines()
file.close()

# Archer2
runCRYSTAL_path = '/work/e05/e05/bcamino/runCRYSTAL'

#Folders
prim_spiral_dir = './data/spiral/primitive/'
super_spiral_dir = './data/spiral/supercell/'
material = 'MnO'

# Primitive cell

In [40]:
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
get_delta_angle(structure_prim_pmg,[-0.01,0.01,0.001],conv_matrix)
get_delta_lattice(structure_prim_pmg,[-0.01,0.01,0.001],conv_matrix)

array([-0.04392561, -0.03953305, -0.03514049, -0.03074793, -0.02635537,
       -0.0219628 , -0.01757024, -0.01317768, -0.00878512, -0.00439256,
       -0.        ,  0.00439256,  0.00878512,  0.01317768,  0.01757024,
        0.0219628 ,  0.02635537,  0.03074793,  0.03514049,  0.03953305,
        0.04392561])

In [51]:
delta_c = get_delta_lattice(structure_prim_pmg,range_lattice,conv_matrix)
delta_a = get_delta_angle(structure_prim_pmg,range_angle,conv_matrix)

input_lattice_labels = np.arange(len(delta_c))-len(delta_c)//2
input_angle_labels = np.arange(len(delta_a))-len(delta_c)//2

In [71]:
supercell_matrix = np.identity(3)*2

delta_c = get_delta_lattice(structure_prim_pmg,range_lattice,conv_matrix)
delta_a = get_delta_angle(structure_prim_pmg,range_angle,conv_matrix)

lattice_parameters = np.zeros((len(delta_c),len(delta_c)))
angle_parameters = np.zeros((len(delta_a),len(delta_a)))

input_lattice_labels = np.arange(len(delta_c))-len(delta_c)//2
input_angle_labels = np.arange(len(delta_a))-len(delta_c)//2



for i,delta_cell in enumerate(delta_c):
    
    structure_prim_pmg_tmp = set_cell(structure_prim_pmg,delta_cell,conv_matrix)

    for j,delta_angle in enumerate(delta_a):
        
        structure_prim_pmg_exp = set_angle(structure_prim_pmg_tmp,delta_angle,conv_matrix)
        #a = np.round(mno_pmg.lattice.a,8)
        #alpha =np.round(mno_pmg.lattice.alpha,6)
               
        lattice_parameters[i][j] = Structure(np.matmul(conv_matrix,structure_prim_pmg_exp.lattice.matrix),[],[]).lattice.a
        angle_parameters[i][j] = Structure(np.matmul(conv_matrix,structure_prim_pmg_exp.lattice.matrix),[],[]).lattice.alpha
        
        # Generate the CRYSTAL gui files
        crystal_gui = cry_pmg2gui(structure_prim_pmg_exp)
        
        
        # Generate the CRYSTAL input files
        crystal_input_freq = copy.deepcopy(prim_crystal_input)
        crystal_input_cpks = copy.deepcopy(prim_crystal_input)
        
        
        #Supercell
        crystal_input_freq.geom_block = ['EXTERNAL\n',
                                 'FREQCALC\n',
                                 'INTENS\n',
                                 'INTPOL\n',
                                 'ENDFREQ\n',
                                 'END\n']
        
        crystal_input_cpks.geom_block = ['EXTERNAL\n'
                                 'CPKS\n',
                                 'MAXCYCLE\n',
                                 '500\n',
                                 'TOLALPHA\n',
                                 '4\n',
                                 'FMIXING\n',
                                 '20\n',
                                 'ENDFREQ\n',
                                 'END\n']
        
        if input_lattice_labels[i] != 0 or input_angle_labels[j] != 0:
            crystal_input_freq.scf_block.insert(-1,'GUESSP\n')
            crystal_input_cpks.scf_block.insert(-1,'GUESSP\n')
        crystal_gui.write_crystal_gui('%s/%s_freq_%s_%s.gui'%(prim_spiral_dir,material,input_lattice_labels[i],input_angle_labels[j]))
        
        crystal_input_freq.write_crystal_input('%s/%s_freq_%s_%s.d12'%(prim_spiral_dir,material,input_lattice_labels[i],input_angle_labels[j]))
        crystal_input_cpks.write_crystal_input('%s/%s_cpks_%s_%s.d12'%(prim_spiral_dir,material,input_lattice_labels[i],input_angle_labels[j]))
        
        # PHONOPY
        symbols = [element(x).symbol for x in structure_prim_pmg_exp.atomic_numbers]
        unitcell = PhonopyAtoms(symbols=symbols,
                                cell=structure_prim_pmg_exp.lattice.matrix,
                                scaled_positions=structure_prim_pmg_exp.frac_coords)
        phonon = Phonopy(unitcell,
                         supercell_matrix)

        phonon.generate_displacements(distance=0.03)
        supercells = [phonon.supercell]
        
        supercells.extend(phonon.supercells_with_displacements)
        
        for k,struct_phonopy in enumerate(supercells):
            
            scel_crystal_input_tmp = copy.deepcopy(scel_crystal_input)
            
            cell = struct_phonopy.get_cell()
            atomic_numbers = struct_phonopy.get_atomic_numbers()
            positions = struct_phonopy.get_scaled_positions()
            
            struct = Structure(cell, atomic_numbers, positions)
                    
            super_disp_gui = cry_pmg2gui(struct,symmetry=True)

            #mno_disp_gui.atom_number = atom_numbers
            
            gui_name = os.path.join(super_spiral_dir,'supercell_%s_%s-00%s.gui'%(input_lattice_labels[i],input_angle_labels[j],k))
            super_disp_gui.write_crystal_gui(gui_name,symm=True)
            
            
            
            #mno_scel_inp.scf_block.insert(-1,'MPP\n')'''
            
            if k == 0:
                if input_lattice_labels[i] != 0 or input_angle_labels[j] != 0:
                    mno_scel_inp.scf_block.insert(-1,'GUESSP\n')
                mno_scel_inp.scf_block.insert(-1,'SAVEPRED\n')
            elif k>0:
                #mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n')
                mno_scel_inp.scf_block.insert(-1,'GUESSYMP\n')
             
            mno_scel_inp.write_crystal_input(join(folder,'supercell_%s_%s-00%s.d12'%(input_lattice_labels[i],input_angle_labels[j],k)))

        
        

### Primitive cell slurm

In [77]:
#### prepare slurm primitive! !
slurm_data_tmp = copy.deepcopy(slurm_data)

x,y = spiral(13, 13)

slurm_data_tmp.append('timeout 2876m %s/Pcry_slurm %s_freq_0_0  &\n'%(runCRYSTAL_path,material))
slurm_data_tmp.append('wait\n')
slurm_data_tmp.append('%s/post_proc_slurm crys %s_freq_0_0 &\n'%(runCRYSTAL_path,material))
slurm_data_tmp.append('wait\n\n')
slurm_data_tmp.append('timeout 2876m %s/Pcry_slurm %s_cpks_0_0 %s_freq_0_0  &\n'%(runCRYSTAL_path,material,material))
slurm_data_tmp.append('wait\n')
slurm_data_tmp.append('%s/post_proc_slurm crys %s_cpks_0_0 &\n'%(runCRYSTAL_path,material))
slurm_data_tmp.append('wait\n\n')
ll = [x for x in range(len(x)-1)][::]

#for i in range(1,len(x)): 
for i in ll:
    #print(i,x[i],y[i])
    slurm_data_tmp.append('timeout 2876m %s/Pcry_slurm %s_freq_%s_%s %s_freq_%s_%s  &\n'% (runCRYSTAL_path,material,x[i+1],y[i+1],material,x[i],y[i]))
    slurm_data_tmp.append('wait\n')
    slurm_data_tmp.append('%s/post_proc_slurm crys %s_freq_%s_%s &\n'% (runCRYSTAL_path,material,x[i+1],y[i+1]))
    slurm_data_tmp.append('wait\n\n')
    
    slurm_data_tmp.append('timeout 2876m %s/Pcry_slurm %s_cpks_%s_%s %s_freq_%s_%s  &\n'% (runCRYSTAL_path,material,x[i+1],y[i+1],material,x[i+1],y[i+1]))
    slurm_data_tmp.append('wait\n')
    slurm_data_tmp.append('%s/post_proc_slurm crys %s_cpks_%s_%s &\n'% (runCRYSTAL_path,material,x[i+1],y[i+1]))
    slurm_data_tmp.append('wait\n\n')
    
with open('%s/%s_crystal_sp.slurm'%(prim_spiral_dir,material), 'w') as file:
    for line in slurm_data_tmp:
        file.writelines(line)

# Supercell - phonons

In [144]:
#CONVERGENCE AFM2
structure = mno_pmg_opt

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()
    
for i in range(2,7):
    
    crystal_input = Crystal_input().from_file('./data/MnO.d12')
    crystal_input.geom_block = [ 'CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '5.31511592 34.22791100\n',
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'SCELPHONO\n',
                                 '%s 0 0\n'%(str(i)),
                                 '0 %s 0\n'%(str(i)),
                                 '0 0 %s\n'%(str(i)),
                                 'FREQCALC\n',
                                 'DISPERSI\n',
                                 'INTERPHESS\n',
                                 '10 10 10\n',
                                 '0\n',
                                 'WANG\n',
                                 '4.313006 0.00000  0.00000\n',
                                 '0.00000 4.326892 0.00000\n',
                                 '0.00000 0.00000 4.326892\n',
                                 'ENDFREQ\n',
                                 'END\n']
    
    
    #spin
    n_atoms = (i**3*4)    
    step = int((i**3*4)/4)
    spin = ['1 +1','3 -1'] 
    for j in np.arange(5,step+2):
        spin.extend(['%s +1'%(j)])
    spin.extend(['%s +1'%(step*2+1)])
    for k in np.arange(step*2+2,step*3+1):
        spin.extend(['%s -1'%(k)])
    spin.extend(['%s +1'%(step*3+1)])
        
    

    spin_input = ' '.join(spin)
    n_spin = int(n_atoms/2)
    crystal_input.scf_block.insert(-1,'ATOMSPIN\n')
    crystal_input.scf_block.insert(-1,'%s\n'%n_spin)
    crystal_input.scf_block.insert(-1,spin_input+'\n')
    
    crystal_input.scf_block.insert(-1,'MPP\n')
    
    sh.copy('./data/primitive/MnO_prim_freq.BORN','./data/convergence/MnO_super%s_phonons.BORN'%str(i))
    
    crystal_input.write_crystal_input('./data/convergence/MnO_super%s_phonons.d12'%str(i))
    
    
    x,y = spiral(13, 13)

    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_super%s_phonons MnO_super%s_phonons &\n'%(str(i),str(i)))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_super%s_phonons &\n'%str(i))
    data.append('wait\n\n')

with open('./data/convergence/MnO_crystal.slurm', 'w') as file:
    for line in data:
        file.writelines(line)    

In [155]:
#CONVERGENCE FM
structure = mno_pmg_opt

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()
    
for i in range(2,7):
    
    crystal_input = Crystal_input().from_file('./data/MnO.d12')
    crystal_input.geom_block = [ 'CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '2.65755796 34.22791100\n',
                                 '2\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 'SCELPHONO\n',
                                 '%s 0 0\n'%(str(i)),
                                 '0 %s 0\n'%(str(i)),
                                 '0 0 %s\n'%(str(i)),
                                 'FREQCALC\n',
                                 'DISPERSI\n',
                                 'INTERPHESS\n',
                                 '10 10 10\n',
                                 '0\n',
                                 'WANG\n',
                                 '4.313006 0.00000  0.00000\n',
                                 '0.00000 4.326892 0.00000\n',
                                 '0.00000 0.00000 4.326892\n',
                                 'ENDFREQ\n',
                                 'END\n']
    
    
    #spin
    n_atoms = (i**3*4)    
    step = int((i**3*2)/2)
    #spin = ['1 +1','3 -1'] 
    spin = ['1 +1']
    for j in np.arange(3,step+2):
        spin.extend(['%s +1'%(j)])
        
    

    spin_input = ' '.join(spin)
    n_spin = int(n_atoms/4)
    crystal_input.scf_block.insert(-1,'ATOMSPIN\n')
    crystal_input.scf_block.insert(-1,'%s\n'%n_spin)
    crystal_input.scf_block.insert(-1,spin_input+'\n')
    
    crystal_input.scf_block.insert(-1,'MPP\n')
    
    sh.copy('./data/primitive/MnO_prim_freq.BORN','./data/convergence/fm/MnO_super%s_phonons.BORN'%str(i))
    
    crystal_input.write_crystal_input('./data/convergence/fm/MnO_super%s_phonons.d12'%str(i))
    
    
    x,y = spiral(13, 13)

    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_super%s_phonons MnO_super%s_phonons &\n'%(str(i),str(i)))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_super%s_phonons &\n'%str(i))
    data.append('wait\n\n')

with open('./data/convergence/fm/MnO_crystal.slurm', 'w') as file:
    for line in data:
        file.writelines(line)    

In [42]:
mno_pmg = cry_gui2pmg(Crystal_gui().read_cry_gui('./data/MnO_prim.gui'))
aa = SpacegroupAnalyzer(mno_pmg).get_primitive_standard_structure()
cry_pmg2gui(aa).write_crystal_gui('./data//test/MnO_symm.gui')

In [36]:
#CONVERGENCE FM PRIMITIVE

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()
    
for i in range(2,7):
    
    crystal_input = Crystal_input().from_file('./data/MnO.d12')
    crystal_input.geom_block = [ 'CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '3.0836682  60.95755790\n',
                                 '2\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.5  0.5  0.5\n',
                                 'SCELPHONO\n',
                                 '%s 0 0\n'%(str(i)),
                                 '0 %s 0\n'%(str(i)),
                                 '0 0 %s\n'%(str(i)),
                                 'FREQCALC\n',
                                 'DISPERSI\n',
                                 'INTERPHESS\n',
                                 '10 10 10\n',
                                 '0\n',
                                 'WANG\n',
                                 '4.313006 0.00000  0.00000\n',
                                 '0.00000 4.326892 0.00000\n',
                                 '0.00000 0.00000 4.326892\n',
                                 'ENDFREQ\n',
                                 'END\n']
    
    
    #spin
    n_atoms = (i**3*4)    
    step = int((i**3*2)/2)
    #spin = ['1 +1','3 -1'] 
    spin = ['1 +1']
    for j in np.arange(3,step+2):
        spin.extend(['%s +1'%(j)])
        
    

    spin_input = ' '.join(spin)
    n_spin = int(n_atoms/4)
    crystal_input.scf_block.insert(-1,'ATOMSPIN\n')
    crystal_input.scf_block.insert(-1,'%s\n'%n_spin)
    crystal_input.scf_block.insert(-1,spin_input+'\n')
    
    crystal_input.scf_block.insert(-1,'MPP\n')
    
    sh.copy('./data/primitive/MnO_prim_freq.BORN','./data/convergence/fm/primitive/MnO_super%s_phonons.BORN'%str(i))
    
    crystal_input.write_crystal_input('./data/convergence/fm/primitive/MnO_super%s_phonons.d12'%str(i))
    
    
    x,y = spiral(13, 13)

    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_super%s_phonons MnO_super%s_phonons &\n'%(str(i),str(i)))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_super%s_phonons &\n'%str(i))
    data.append('wait\n\n')

with open('./data/convergence/fm/MnO_crystal.slurm', 'w') as file:
    for line in data:
        file.writelines(line)    

In [146]:
#HERE NEW PRIMITIVE BORN+DIELTENS!!
structure = mno_pmg_opt
angle_step = 0.821316026348/6
cell_step = (4.392560947738861*0.02)/6
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
num_steps = 6

lattice_parameters = np.zeros((13,13))
angle_parameters = np.zeros((13,13))

#supercell_matrix = np.identity(3)*4
supercell_matrix = np.identity(3)*2


for i,delta_c in enumerate(np.arange(-num_steps,num_steps+1)):
    
    delta_cell = delta_c*cell_step
    #print(i,delta_cell)
    mno_pmg_tmp = set_cell(mno_pmg_opt,delta_cell,conv_matrix)
    #print(mno_pmg_tmp.lattice.a)
    for j,delta_a in enumerate(np.arange(-num_steps,num_steps+1)):

        delta_angle = delta_a*angle_step
        
        mno_pmg = set_angle(mno_pmg_tmp,delta_angle,conv_matrix)
        a = np.round(mno_pmg.lattice.a,8)
        alpha =np.round(mno_pmg.lattice.alpha,6)
        
        
        lattice_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.a
        angle_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.alpha
        
        crystal_input_freq = Crystal_input().from_file('./data/MnO_prim.d12')
        crystal_input_cpks = Crystal_input().from_file('./data/MnO_prim.d12')
        
        
        #Supercell
        crystal_input_freq.geom_block = ['CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '%s %s\n'%(a,alpha),
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'FREQCALC\n',
                                 'INTENS\n',
                                 'INTPOL\n',
                                 'ENDFREQ\n',
                                 'END\n']
        
        crystal_input_cpks.geom_block = ['CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '%s %s\n'%(a,alpha),
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'CPKS\n',
                                 'MAXCYCLE\n',
                                 '500\n',
                                 'TOLALPHA\n',
                                 '4\n',
                                 'FMIXING\n',
                                 '20\n',
                                 'ENDFREQ\n',
                                 'END\n']
        
        crystal_input_cpks.scf_block.insert(-1,'ATOMSPIN\n')
        crystal_input_cpks.scf_block.insert(-1,'2\n')
        crystal_input_cpks.scf_block.insert(-1,'1 +1 3 -1\n')
        
        crystal_input_freq.scf_block.insert(-1,'ATOMSPIN\n')
        crystal_input_freq.scf_block.insert(-1,'2\n')
        crystal_input_freq.scf_block.insert(-1,'1 +1 3 -1\n')


        crystal_input_freq.write_crystal_input('./data/spiral/primitive/MnO_freq_%s_%s.d12'%(delta_c,delta_a))
        crystal_input_cpks.write_crystal_input('./data/spiral/primitive/MnO_cpks_%s_%s.d12'%(delta_c,delta_a))
        
        
        
        
        
        

In [23]:
#HERE NEW PRIMITIVE BORN+DIELTENS FM!!
structure = mno_pmg_opt
angle_step = 0.821316026348/6
cell_step = (4.392560947738861*0.02)/6
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
num_steps = 6

lattice_parameters = np.zeros((13,13))
angle_parameters = np.zeros((13,13))

#supercell_matrix = np.identity(3)*4
supercell_matrix = np.identity(3)*2


for i,delta_c in enumerate(np.arange(-num_steps,num_steps+1)):
    
    delta_cell = delta_c*cell_step
    #print(i,delta_cell)
    mno_pmg_tmp = set_cell(mno_pmg_opt,delta_cell,conv_matrix)
    #print(mno_pmg_tmp.lattice.a)
    for j,delta_a in enumerate(np.arange(-num_steps,num_steps+1)):

        delta_angle = delta_a*angle_step
        
        mno_pmg = set_angle(mno_pmg_tmp,delta_angle,conv_matrix)
        a = np.round(mno_pmg.lattice.a,8)
        alpha =np.round(mno_pmg.lattice.alpha,6)
        
        
        lattice_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.a
        angle_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.alpha
        
        crystal_input_freq = Crystal_input().from_file('./data/MnO_fm_prim.d12')
        crystal_input_cpks = Crystal_input().from_file('./data/MnO_fm_prim.d12')
        
        
        #Supercell
        crystal_input_freq.geom_block = ['CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '%s %s\n'%(a,alpha),
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'FREQCALC\n',
                                 'INTENS\n',
                                 'INTPOL\n',
                                 'ENDFREQ\n',
                                 'END\n']
        
        crystal_input_cpks.geom_block = ['CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '%s %s\n'%(a,alpha),
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'CPKS\n',
                                 'MAXCYCLE\n',
                                 '500\n',
                                 'TOLALPHA\n',
                                 '4\n',
                                 'FMIXING\n',
                                 '20\n',
                                 'ENDFREQ\n',
                                 'END\n']
        
        crystal_input_cpks.scf_block.insert(-1,'ATOMSPIN\n')
        crystal_input_cpks.scf_block.insert(-1,'2\n')
        crystal_input_cpks.scf_block.insert(-1,'1 +1 3 +1\n')
        
        crystal_input_freq.scf_block.insert(-1,'ATOMSPIN\n')
        crystal_input_freq.scf_block.insert(-1,'2\n')
        crystal_input_freq.scf_block.insert(-1,'1 +1 3 +1\n')


        crystal_input_freq.write_crystal_input('./data/spiral/primitive/fm/MnO_freq_%s_%s.d12'%(delta_c,delta_a))
        crystal_input_cpks.write_crystal_input('./data/spiral/primitive/fm/MnO_cpks_%s_%s.d12'%(delta_c,delta_a))
        
        
        
        
        
        

In [28]:
#### prepare slurm primitive! !

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()

x,y = spiral(13, 13)

data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_freq_0_0  &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_freq_0_0 &\n')
data.append('wait\n\n')
data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_cpks_0_0 MnO_freq_0_0  &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_cpks_0_0 &\n')
data.append('wait\n\n')
ll = [x for x in range(len(x)-1)][::]

#for i in range(1,len(x)): 
for i in ll:
    #print(i,x[i],y[i])
    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_freq_%s_%s MnO_freq_%s_%s  &\n'% (x[i+1],y[i+1],x[i],y[i]))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_freq_%s_%s &\n'% (x[i+1],y[i+1]))
    data.append('wait\n\n')
    
    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_cpks_%s_%s MnO_freq_%s_%s  &\n'% (x[i+1],y[i+1],x[i+1],y[i+1]))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_cpks_%s_%s &\n'% (x[i+1],y[i+1]))
    data.append('wait\n\n')
    
with open('./data/spiral/primitive/fm/MnO_crystal_sp.slurm', 'w') as file:
    for line in data:
        file.writelines(line)

In [58]:
#HERE NEW AFM!!
structure = mno_pmg_opt
angle_step = 0.821316026348/6
cell_step = (4.392560947738861*0.02)/6
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
num_steps = 5

lattice_parameters = np.zeros((13,13))
angle_parameters = np.zeros((13,13))

#supercell_matrix = np.identity(3)*4
supercell_matrix = np.identity(3)*2


for i,delta_c in enumerate(np.arange(-num_steps,num_steps+1)):
    
    delta_cell = delta_c*cell_step
    #print(i,delta_cell)
    mno_pmg_tmp = set_cell(mno_pmg_opt,delta_cell,conv_matrix)
    #print(mno_pmg_tmp.lattice.a)
    for j,delta_a in enumerate(np.arange(-num_steps,num_steps+1)):

        delta_angle = delta_a*angle_step
        
        mno_pmg = set_angle(mno_pmg_tmp,delta_angle,conv_matrix)
        a = np.round(mno_pmg.lattice.a,8)
        alpha =np.round(mno_pmg.lattice.alpha,6)
        
        
        lattice_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.a
        angle_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.alpha
        
        crystal_input = Crystal_input().from_file('./data/MnO_spin4.d12')
        crystal_output_prim = Crystal_output().read_cry_output('./data/spiral/primitive/MnO_cpks_%s_%s.out'%(delta_c,delta_a))
        dielectric_tensor = crystal_output_prim.get_dielectric_tensor()
        #Supercell
        crystal_input.geom_block = ['CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '%s %s\n'%(a,alpha),
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'SCELPHONO\n',
                                 '4 0 0\n',
                                 '0 4 0\n',
                                 '0 0 4\n',
                                 'FREQCALC\n',
                                 'DISPERSI\n',
                                 'INTERPHESS\n',
                                 '10 10 10\n',
                                 '0\n',
                                 'WANG\n',
                                 '%s 0.00000  0.00000\n'%dielectric_tensor[0],
                                 '0.00000 %s 0.00000\n'%dielectric_tensor[1],
                                 '0.00000 0.00000 %s\n'%dielectric_tensor[2],
                                 'ENDFREQ\n',
                                 'END\n']
        
        folder = './data/spiral/'
        sh.copy('./data/spiral/primitive/MnO_freq_%s_%s.BORN'%(delta_c,delta_a),'./data/spiral/afm/MnO_phonons_%s_%s.BORN'%(delta_c,delta_a))
        crystal_input.write_crystal_input('./data/spiral/afm/MnO_phonons_%s_%s.d12'%(delta_c,delta_a))
        
        
        
        
        
        

In [59]:
#### prepare slurm primitive! !

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()

x,y = spiral(13, 13)

data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_phonons_0_0 MnO_phonons_0_0 \n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_phonons_0_0 &\n')
data.append('wait\n\n')

ll = [x for x in range(len(x)-1)][::]

#for i in range(1,len(x)): 
for i in ll:
    data.append('cp MnO_phonons_%s_%s.f9 MnO_phonons_%s_%s.f9 &\n'% (x[i],y[i],x[i+1],y[i+1]))
    data.append('wait\n')
    #print(i,x[i],y[i])
    
    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_phonons_%s_%s MnO_phonons_%s_%s  &\n'% (x[i+1],y[i+1],x[i+1],y[i+1]))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_phonons_%s_%s &\n'% (x[i+1],y[i+1]))
    data.append('wait\n\n')
    
with open('./data/spiral/afm/MnO_crystal_sp.slurm', 'w') as file:
    for line in data:
        file.writelines(line)

In [60]:
#HERE NEW FM!!
structure = mno_pmg_opt
angle_step = 0.821316026348/6
cell_step = (4.392560947738861*0.02)/6
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
num_steps = 5

lattice_parameters = np.zeros((13,13))
angle_parameters = np.zeros((13,13))

#supercell_matrix = np.identity(3)*4
supercell_matrix = np.identity(3)*2


for i,delta_c in enumerate(np.arange(-num_steps,num_steps+1)):
    
    delta_cell = delta_c*cell_step
    print(i,delta_cell)
    mno_pmg_tmp = set_cell(mno_pmg_opt,delta_cell,conv_matrix)
    #print(mno_pmg_tmp.lattice.a)
    for j,delta_a in enumerate(np.arange(-num_steps,num_steps+1)):

        delta_angle = delta_a*angle_step
        
        mno_pmg = set_angle(mno_pmg_tmp,delta_angle,conv_matrix)
        a = np.round(mno_pmg.lattice.a,8)
        alpha =np.round(mno_pmg.lattice.alpha,6)
        
        
        lattice_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.a
        angle_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.alpha
        
        crystal_input = Crystal_input().from_file('./data/MnO_fm_spin4.d12')
        crystal_output_prim = Crystal_output().read_cry_output('./data/spiral/primitive/fm/MnO_cpks_%s_%s.out'%(delta_c,delta_a))
        dielectric_tensor = crystal_output_prim.get_dielectric_tensor()
        #Supercell
        crystal_input.geom_block = ['CRYSTAL\n',
                                 '0 1 0\n',
                                 '160\n',
                                 '%s %s\n'%(a,alpha),
                                 '4\n',
                                 '25  0.00  0.00  0.00\n',
                                 '8  0.25  0.25  0.25\n',
                                 '25  0.50  0.50  0.50\n',
                                 '8  0.75  0.75  0.75\n',
                                 'SCELPHONO\n',
                                 '4 0 0\n',
                                 '0 4 0\n',
                                 '0 0 4\n',
                                 'FREQCALC\n',
                                 'DISPERSI\n',
                                 'INTERPHESS\n',
                                 '10 10 10\n',
                                 '0\n',
                                 'WANG\n',
                                 '%s 0.00000  0.00000\n'%dielectric_tensor[0],
                                 '0.00000 %s 0.00000\n'%dielectric_tensor[1],
                                 '0.00000 0.00000 %s\n'%dielectric_tensor[2],
                                 'ENDFREQ\n',
                                 'END\n']
        
        folder = './data/spiral/'
        sh.copy('./data/spiral/primitive/fm/MnO_freq_%s_%s.BORN'%(delta_c,delta_a),'./data/spiral/fm/MnO_phonons_%s_%s.BORN'%(delta_c,delta_a))
        crystal_input.write_crystal_input('./data/spiral/fm/MnO_phonons_%s_%s.d12'%(delta_c,delta_a))
                
        

0 -0.07320934912898103
1 -0.058567479303184816
2 -0.04392560947738861
3 -0.029283739651592408
4 -0.014641869825796204
5 0.0
6 0.014641869825796204
7 0.029283739651592408
8 0.04392560947738861
9 0.058567479303184816
10 0.07320934912898103


In [61]:
#### prepare slurm NEW! !

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()

x,y = spiral(13, 13)

data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_phonons_0_0 MnO_phonons_0_0\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_phonons_0_0\n')
ll = [x for x in range(len(x)-1)][::]
#for i in range(1,len(x)): 
for i in ll:
    #print(i,x[i],y[i])
    data.append('cp MnO_phonons_%s_%s.f9 MnO_phonons_%s_%s.f9\n'% (x[i],y[i],x[i+1],y[i+1]))

    
    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm MnO_phonons_%s_%s MnO_phonons_%s_%s\n'% (x[i+1],y[i+1],x[i+1],y[i+1]))
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys MnO_phonons_%s_%s\n'% (x[i+1],y[i+1]))
    
    
    
with open('./data/spiral/afm/MnO_crystal_sp.slurm', 'w') as file:
    for line in data:
        file.writelines(line)

# PHONOPY

<b>PRIMITIVE</b>:

- loop over the same expansion cpks/freq

<b>FIXINDEX</b>:

- Use the geometry input of the primitive cell (CRYSTAL,...)
- SUPERCELL
- ATOMINSE (reference geometry (central point)
- ATOMREMO (n atoms in the supercell)
- BREAKSYMM?
- EXTERNAL in the 2nd GEOM block
    
<b>GUESS</b>:

- None for the initial structure
- GUESSYMP for the displacements from the undisplaced one
- GUESSP for the new point on the grid from the previous undisplaced point

In [96]:
#HERE OLD NEW PHONOPY

from CRYSTALpy import crystal_io
structure = mno_pmg_opt
angle_step = 0.821316026348/6
cell_step = (4.392560947738861*0.02)/6
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
num_steps = 5

lattice_parameters = np.zeros((13,13))
angle_parameters = np.zeros((13,13))

#supercell_matrix = np.identity(3)*4
supercell_matrix = np.identity(3)*2


for i,delta_c in enumerate(np.arange(-num_steps,num_steps+1)):   
    delta_cell = delta_c*cell_step
    #print(delta_cell)
    mno_pmg_tmp = set_cell(mno_pmg_opt,delta_cell,conv_matrix)
    #display(Structure(np.matmul(conv_matrix,mno_pmg_tmp.lattice.matrix),[],[]).lattice.a)
    #print(delta_c,mno_pmg_tmp.lattice.a,mno_pmg_tmp.lattice.alpha)
    #print()
    for j,delta_a in enumerate(np.arange(-num_steps,num_steps+1)):

        delta_angle = delta_a*angle_step
        
        mno_pmg = set_angle(mno_pmg_tmp,delta_angle,conv_matrix)

        lattice_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.a
        angle_parameters[i][j] = Structure(np.matmul(conv_matrix,mno_pmg.lattice.matrix),[],[]).lattice.alpha
        
        folder = './data/phonopy/'
        
        if not os.path.exists(folder):           
            os.mkdir(folder)
        
        # WRITE THE PRIMITIVE CELL STRUCTURE FOR BORN CHARGES CALCULATION:
        #mno_gui = cry_pmg2gui(mno_pmg)
        #gui_name = os.path.join(folder,'primitive_%s_%s.gui'%(delta_c,delta_a))
        #mno_gui.write_crystal_gui(gui_name,symm=True)
        
        mno_inp = Crystal_input().from_file(os.path.join('./data','MnO_prim.d12'))
        mno_inp.write_crystal_input(join(folder,'primitive_%s_%s.d12'%(delta_c,delta_a)))
        
        # WRITE STRUCTURE WITH PHONOPY-API
        
        symbols = [element(x).symbol for x in mno_pmg.atomic_numbers]
        unitcell = PhonopyAtoms(symbols=symbols,
                                cell=mno_pmg.lattice.matrix,
                                scaled_positions=mno_pmg.frac_coords)
        phonon = Phonopy(unitcell,
                         supercell_matrix=[[4, 0, 0], [0, 4, 0], [0, 0, 4]])
                         #primitive_matrix=[[1., 0.0, 0.0],
                              #             [0.0, 1., 0.0],
                              #             [0.0, 0.0, 1.]])
        phonon.generate_displacements(distance=0.03)
        supercells = [phonon.supercell]
        
        supercells.extend(phonon.supercells_with_displacements)

        
        #displaced structures
        for k,struct_phonopy in enumerate(supercells):
            
            mno_scel_inp = Crystal_input().from_file(os.path.join('./data','MnO_spin4_phonopy.d12'))
            
            cell = struct_phonopy.get_cell()
            atomic_numbers = struct_phonopy.get_atomic_numbers()
            positions = struct_phonopy.get_scaled_positions()
            
            struct = Structure(cell, atomic_numbers, positions)
            
            atom_numbers = list(struct.atomic_numbers)
            range_scel = int(struct.num_sites/mno_pmg.num_sites)
            
            for m in range(range_scel):
                struct.replace(m,1)
            
            spin = []
            for n in np.where(np.array(struct.atomic_numbers)==1)[0]:
                spin.extend(['%s +1'%(n+1)])
            for n in np.where(np.array(struct.atomic_numbers)==25)[0]:
                spin.extend(['%s -1'%(n+1)])
            
            n_spin = len(np.where(np.array(struct.atomic_numbers)==1)[0])+\
                len(np.where(np.array(struct.atomic_numbers)==25)[0])
            spin = ' '.join(spin)  
            
            
            mno_disp_gui = cry_pmg2gui(struct,symmetry=True)

            mno_disp_gui.atom_number = atom_numbers
            
            gui_name = os.path.join(folder,'supercell_%s_%s-00%s.gui'%(delta_c,delta_a,k))
            mno_disp_gui.write_crystal_gui(gui_name,symm=True)
            
            #mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n') 
            mno_scel_inp.scf_block.insert(-1,'ATOMSPIN\n')
            mno_scel_inp.scf_block.insert(-1,'%s\n'%n_spin)
            mno_scel_inp.scf_block.insert(-1,spin+'\n')
            
            #mno_scel_inp.scf_block.insert(-1,'MPP\n')'''
            
            if k == 0:
                if delta_c != 0 or delta_a != 0:
                    mno_scel_inp.scf_block.insert(-1,'GUESSP\n')
                mno_scel_inp.scf_block.insert(-1,'SAVEPRED\n')
            elif k>0:
                mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n')
                mno_scel_inp.scf_block.insert(-1,'GUESSYMP\n')
             
            mno_scel_inp.write_crystal_input(join(folder,'supercell_%s_%s-00%s.d12'%(delta_c,delta_a,k)))
        
        

In [73]:
#### prepare slurm

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()

x,y = spiral(11,11)

data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_0_0-000 &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_0_0-000 &\n')
data.append('wait\n\n')
data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_0_0-001 supercell_0_0-000 &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_0_0-001 &\n')
data.append('wait\n\n')
data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_0_0-002 supercell_0_0-000 &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_0_0-002 &\n')
data.append('wait\n\n')

ll = [x for x in range(len(x)-1)][::]
#for i in range(1,len(x)): 
for i in ll:
    print(i,x[i],y[i])
    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_%s_%s-000 supercell_%s_%s-000 &\n'% (x[i+1],y[i+1],x[i],y[i]))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_%s_%s-000 &\n'% (x[i],y[i]))
    data.append('wait\n\n')
    for j in range(1,3):
        data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_%s_%s-00%s supercell_%s_%s-000 &\n'% (x[i+1],y[i+1],j,x[i+1],y[i+1]))
        data.append('wait\n')
        data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_%s_%s-00%s &\n'% (x[i],y[i],j))
        data.append('wait\n\n')
    data.append('')

with open('./data/phonopy/MnO_crystal.slurm', 'w') as file:
    for line in data:
        file.writelines(line)

0 0 0
1 1 0
2 1 1
3 0 1
4 -1 1
5 -1 0
6 -1 -1
7 0 -1
8 1 -1
9 2 -1
10 2 0
11 2 1
12 2 2
13 1 2
14 0 2
15 -1 2
16 -2 2
17 -2 1
18 -2 0
19 -2 -1
20 -2 -2
21 -1 -2
22 0 -2
23 1 -2
24 2 -2
25 3 -2
26 3 -1
27 3 0
28 3 1
29 3 2
30 3 3
31 2 3
32 1 3
33 0 3
34 -1 3
35 -2 3
36 -3 3
37 -3 2
38 -3 1
39 -3 0
40 -3 -1
41 -3 -2
42 -3 -3
43 -2 -3
44 -1 -3
45 0 -3
46 1 -3
47 2 -3
48 3 -3
49 4 -3
50 4 -2
51 4 -1
52 4 0
53 4 1
54 4 2
55 4 3
56 4 4
57 3 4
58 2 4
59 1 4
60 0 4
61 -1 4
62 -2 4
63 -3 4
64 -4 4
65 -4 3
66 -4 2
67 -4 1
68 -4 0
69 -4 -1
70 -4 -2
71 -4 -3
72 -4 -4
73 -3 -4
74 -2 -4
75 -1 -4
76 0 -4
77 1 -4
78 2 -4
79 3 -4
80 4 -4
81 5 -4
82 5 -3
83 5 -2
84 5 -1
85 5 0
86 5 1
87 5 2
88 5 3
89 5 4
90 5 5
91 4 5
92 3 5
93 2 5
94 1 5
95 0 5
96 -1 5
97 -2 5
98 -3 5
99 -4 5
100 -5 5
101 -5 4
102 -5 3
103 -5 2
104 -5 1
105 -5 0
106 -5 -1
107 -5 -2
108 -5 -3
109 -5 -4
110 -5 -5
111 -4 -5
112 -3 -5
113 -2 -5
114 -1 -5
115 0 -5
116 1 -5
117 2 -5
118 3 -5
119 4 -5


In [75]:
view(cry_gui2ase(Crystal_gui().read_cry_gui('../anna/part_optimisation/li001-li2S110_sym4L6L.gui')))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [77]:
aa = cry_gui2pmg(Crystal_gui().read_cry_gui('../anna/part_optimisation/li001-li2S110_sym4L6L.gui'))

In [91]:
ll = []
lll = []
for i in range(aa.num_sites):
    if aa.cart_coords[i,2] < 5:
        ll.append(str(i+1))
    else:
        lll.append(str(i+1))

In [94]:
' '.join(ll)

'1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180'

In [225]:
#### prepare slurm

slurm_file = './data/slurm_file.slurm'

file = open(slurm_file, 'r')
data = file.readlines()
file.close()

x,y = spiral(3, 3)

data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_0_0-000 supercell &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_0_0-000 &\n')
data.append('wait\n\n')
data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_0_0-001 supercell_0_0-000 &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_0_0-001 &\n')
data.append('wait\n\n')
data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_0_0-002 supercell_0_0-000 &\n')
data.append('wait\n')
data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_0_0-002 &\n')
data.append('wait\n\n')

for i in range(1,len(x)):    
    data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_%s_%s-000 supercell_%s_%s-000 &\n'% (x[i],y[i],x[i-1],y[i-1]))
    data.append('wait\n')
    data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_%s_%s-000 &\n'% (x[i],y[i]))
    data.append('wait\n\n')
    for j in range(1,3):
        data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm supercell_%s_%s-00%s supercell_%s_%s-000 &\n'% (x[i],y[i],j,x[i],y[i]))
        data.append('wait\n')
        data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys supercell_%s_%s-00%s &\n'% (x[i],y[i],j))
        data.append('wait\n\n')
    data.append('')

with open('./data/supercell_4/supercell.slurm', 'w') as file:
    for line in data:
        file.writelines(line)

7
6
5
4
3
2
1
0


In [12]:
#FIXINDEX
gui_file = Crystal_gui().read_cry_gui('./data/fixindex/supercell_0_0-000.gui')
a = cry_gui2pmg(gui_file)
for i,line in enumerate(np.round(a.frac_coords,6)):
    print(a.atomic_numbers[i],' '.join([str(x) for x in line]))
SpacegroupAnalyzer(a).get_space_group_number()

25 0.0 0.0 0.0
8 -0.375 0.125 0.125
25 0.25 0.25 0.25
8 -0.125 -0.125 0.375
25 -0.5 -0.0 -0.0
25 -0.5 -0.0 -0.5
25 -0.5 -0.5 -0.0
25 -0.5 -0.5 -0.5
25 -0.0 -0.0 -0.5
8 -0.375 0.125 -0.375
8 -0.375 -0.375 0.125
8 -0.375 -0.375 -0.375
8 0.125 0.125 0.125
8 0.125 0.125 -0.375
8 0.125 -0.375 0.125
8 0.125 -0.375 -0.375
25 -0.0 -0.5 -0.0
25 0.25 0.25 -0.25
25 0.25 -0.25 0.25
25 0.25 -0.25 -0.25
25 -0.25 0.25 0.25
25 -0.25 0.25 -0.25
25 -0.25 -0.25 0.25
25 -0.25 -0.25 -0.25
25 -0.0 -0.5 -0.5
8 -0.125 -0.125 -0.125
8 -0.125 0.375 0.375
8 -0.125 0.375 -0.125
8 0.375 -0.125 0.375
8 0.375 -0.125 -0.125
8 0.375 0.375 0.375
8 0.375 0.375 -0.125


166

In [13]:
5.41511592*2

10.83023184

In [59]:
def spiral(X, Y):
    x = y = 0
    dx = 0
    dy = -1
    x_list = []
    y_list = []
    for i in range(max(X, Y)**2):
        if (-X/2 < x <= X/2) and (-Y/2 < y <= Y/2):
            #print (x, y)
            x_list.append(x)
            y_list.append(y)
        if x == y or (x < 0 and x == -y) or (x > 0 and x == 1-y):
            dx, dy = -dy, dx
        x, y = x+dx, y+dy
    return x_list, y_list

In [250]:
angle = 1.
shift = 0.
to_rad = np.pi/180
angle = angle*to_rad

#CELL

#Conversion matrices
conv_matrix = np.array([[1.5, -.5, -.5],[-.5, 1.5, -.5],[-.5,-.5,1.5]])
conv_matrix_inv = np.linalg.inv(conv_matrix)
cubic_lattice = np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix)

#New cubic lattice

len_A = np.linalg.norm(cubic_lattice[0,:])
len_B = np.linalg.norm(cubic_lattice[1,:])
len_C = np.linalg.norm(cubic_lattice[2,:])

cubic_lattice[0,:] = (cubic_lattice[0,:]/len_A) * (len_A+shift)
cubic_lattice[1,:] = (cubic_lattice[1,:]/len_B) * (len_B+shift)
cubic_lattice[2,:] = (cubic_lattice[2,:]/len_C) * (len_C+shift)
print(cubic_lattice)

#New prim lattice
prim_lattice = np.matmul(conv_matrix_inv,cubic_lattice)

A_prim = prim_lattice[0,:]
B_prim = prim_lattice[1,:]
C_prim = prim_lattice[2,:]

len_A_prim = np.linalg.norm(A_prim)
len_B_prim = np.linalg.norm(B_prim)
len_C_prim = np.linalg.norm(C_prim)


A = cubic_lattice[0,:]
B = cubic_lattice[1,:]
C = cubic_lattice[2,:]

len_A = np.linalg.norm(A)
len_B = np.linalg.norm(B)
len_C = np.linalg.norm(C)


A_norm = A/len_A
B_norm = B/len_B
C_norm = C/len_C

alpha = np.arccos(np.dot(B_norm,C_norm))*(180/np.pi)
beta = np.arccos(np.dot(A_norm,C_norm))*(180/np.pi)
gamma = np.arccos(np.dot(A_norm,B_norm))*(180/np.pi)

vector_sum = A_norm+B_norm+C_norm

len_vector_sum = np.linalg.norm(vector_sum)
vector_sum_norm = vector_sum/len_vector_sum

D_a = np.cross(np.cross(A_norm,vector_sum_norm),A_norm)
D_b = np.cross(np.cross(B_norm,vector_sum_norm),B_norm)
D_c = np.cross(np.cross(C_norm,vector_sum_norm),C_norm)

len_D_a = np.linalg.norm(D_a)
len_D_b = np.linalg.norm(D_b)
len_D_c = np.linalg.norm(D_c)

D_a_norm = D_a/len_D_a
D_b_norm = D_b/len_D_b
D_c_norm = D_c/len_D_c

alpha_t = np.arccos(np.dot(A_norm,vector_sum_norm))*(180/np.pi)
beta_t = np.arccos(np.dot(B_norm,vector_sum_norm))*(180/np.pi)
gamma_t = np.arccos(np.dot(C_norm,vector_sum_norm))*(180/np.pi)

len_vector_sum_p = np.sqrt(3 + 2*(np.cos((alpha+delta_angle)*to_rad) +
                                    np.cos((beta+delta_angle)*to_rad) +
                                    np.cos((gamma+delta_angle)*to_rad)))

alpha_p = (np.arccos((  1+ np.cos((beta+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
beta_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((gamma+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg
gamma_p = (np.arccos((  1+ np.cos((alpha+delta_angle)*to_rad) + np.cos((beta+delta_angle)*to_rad) ) /len_vector_sum_p))*to_deg

delta_alpha_t = (alpha_t-alpha_p)*to_rad
delta_beta_t = (beta_t-beta_p)*to_rad
delta_gamma_t = (gamma_t-gamma_p)*to_rad

A1 = (np.cos(angle)*A_norm + np.sin(angle)*D_a_norm) #* len_A
B1 = (np.cos(angle)*B_norm + np.sin(angle)*D_b_norm) #* len_B
C1 = (np.cos(angle)*C_norm + np.sin(angle)*D_c_norm) #* len_C

new_lattice_conv = np.array([A1,B1,C1])


new_lattice_prim = np.matmul(conv_matrix_inv,new_lattice_conv)

len_A_prim_new = np.linalg.norm(new_lattice_prim[0,:])
len_B_prim_new = np.linalg.norm(new_lattice_prim[1,:])
len_C_prim_new = np.linalg.norm(new_lattice_prim[2,:])


new_lattice_prim[0,:] = (new_lattice_prim[0,:]/ len_A_prim_new) * (len_A_prim)
new_lattice_prim[1,:] = (new_lattice_prim[1,:]/len_B_prim_new) * (len_B_prim)
new_lattice_prim[2,:] = (new_lattice_prim[2,:]/len_C_prim_new)  *(len_C_prim)

print(np.linalg.norm(new_lattice_prim[0,:]))
print(np.matmul(conv_matrix,new_lattice_prim))
aa = Structure(new_lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)
Structure(np.matmul(conv_matrix,new_lattice_prim),[],[])

[[ 3.61212438  0.          2.49942977]
 [-1.80606219  3.12819147  2.49942977]
 [-1.80606219 -3.12819147  2.49942977]]
5.315115920002191
[[ 3.49514025  0.          2.50980357]
 [-1.74757012  3.02688024  2.50980357]
 [-1.74757012 -3.02688024  2.50980357]]


Structure Summary
Lattice
    abc : 4.302919860799546 4.30291986079296 4.30291986079296
 angles : 89.40858791294572 89.40858791308888 89.40858791308888
 volume : 79.65642917248634
      A : 3.4951402478580524 0.0 2.5098035732435178
      B : -1.7475701239312018 3.026880244427586 2.509803573239066
      C : -1.7475701239312018 -3.026880244427586 2.509803573239066

In [252]:
Structure(np.matmul(conv_matrix,mno_pmg_opt.lattice.matrix),[],[])

Structure Summary
Lattice
    abc : 4.392560947738861 4.39256094773121 4.39256094773121
 angles : 90.821316026348 90.8213160265301 90.8213160265301
 volume : 84.72629516865366
      A : 3.612124378639 0.0 2.499429765535
      B : -1.8060621893189999 3.12819147352 2.499429765535
      C : -1.8060621893189999 -3.12819147352 2.499429765535

In [97]:
mno_gui_opt = Crystal_gui().read_cry_gui('./isa/mno/MnO.gui')
mno_pmg_opt = cry_gui2pmg(mno_gui_opt)

a,b,c = mno_pmg_opt.lattice.abc
alpha, beta, gamma = mno_pmg_opt.lattice.angles 

directory_opt = './isa/MnO/super_6/'

#for i,disp in enumerate([0.,0.03,0.3]):
for i,disp in enumerate([0.,0.3]):
    
    #for j, angle in enumerate([0.,0.15,1.5]):
    for j, angle in enumerate([0.]):
        
        lattice = mno_pmg_opt.lattice.matrix + mno_pmg_opt.lattice.matrix*disp
        #lattice = Lattice.from_parameters(a+disp, b+disp, c+disp, alpha+angle, beta+angle, gamma+angle)
        
        #lattice = mno_pmg_opt.lattice.matrix
        
        mno_pmg = Structure(lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)
        #mno_pmg_symm = SpacegroupAnalyzer(mno_pmg_tmp).get_symmetrized_structure()
        #mno_pmg = Structure(mno_pmg_symm.lattice.matrix,mno_pmg_opt.atomic_numbers,mno_pmg_symm.frac_coords)
        #display(mno_pmg_symm.lattice.matrix,mno_pmg)
        #print(mno_pmg,'\n',Structure(mno_pmg_symm.lattice.matrix,mno_pmg_opt.atomic_numbers,mno_pmg_symm.frac_coords))
        
        disp_str = get_displaced_structures(mno_pmg,atom_disp=0.01, supercell_matrix=supercell_matrix)
        mno_scel_atomic_numbers = disp_str[0].atomic_numbers

        '''norm = np.array([np.linalg.norm(mno_pmg.lattice.matrix[:,:],axis=1)]*3)
        lattice_norm = mno_pmg.lattice.matrix/norm
        new_lattice = lattice_norm-np.array([np.sum(lattice_norm,axis=0)*angle]*3)
        new_lattice_norm = new_lattice/np.array([np.linalg.norm(new_lattice[:,:],axis=1)]*3)
        new_lattice = new_lattice_norm*norm
        Structure(new_lattice,mno_pmg_opt.atomic_numbers,mno_pmg_opt.frac_coords)'''
        
        
    
        directory = './isa/MnO/super_6/%s_%s'%(i,j)
        print(i,j,directory)
        try: 
            os.mkdir(directory) 
        except OSError as error: 
            pass  
            
        range_scel = int(disp_str[0].num_sites/mno_pmg.num_sites)
        for m in range(len(disp_str)):
            for j in range(range_scel):
                disp_str[m].replace(j,1)

        spin = []
        for n in np.where(np.array(disp_str[0].atomic_numbers)==1)[0]:
            spin.extend(['%s +1'%(n+1)])
        for n in np.where(np.array(disp_str[0].atomic_numbers)==25)[0]:
            spin.extend(['%s -1'%(n+1)])

        n_spin = len(np.where(np.array(disp_str[0].atomic_numbers)==1)[0])+\
            len(np.where(np.array(disp_str[0].atomic_numbers)==25)[0])
        spin = ' '.join(spin)       

        mno_scel = cry_pmg2gui(disp_str[0],symmetry=True)
        mno_scel.atom_number = mno_scel_atomic_numbers

        write_crystal_gui(join(directory,'supercell_symm.gui'),mno_scel)

        mno_scel = cry_pmg2gui(disp_str[0],symmetry=False)
        mno_scel.atom_number = mno_scel_atomic_numbers

        write_crystal_gui(join(directory,'supercell.gui'),mno_scel)


        mno_scel_inp = Crystal_input().from_file(join(directory_opt,'MnO.d12'))


        #INPUT FILES
        mno_scel_inp = Crystal_input().from_file(join(directory_opt,'MnO.d12'))
        mno_scel_inp.scf_block.insert(-1,'ATOMSPIN\n')
        mno_scel_inp.scf_block.insert(-1,'%s\n'%n_spin)
        mno_scel_inp.scf_block.insert(-1,spin+'\n')
        mno_scel_inp.scf_block.insert(-1,'SAVEPRED\n')

        write_crystal_input(join(directory,'supercell_symm.d12'),mno_scel_inp)
        
        mno_scel_inp.scf_block.remove('SAVEPRED\n')
        
        mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n')
        mno_scel_inp.scf_block.insert(-1,'GUESSYMP\n')
        mno_scel_inp.scf_block.insert(-1,'MPP\n')

        write_crystal_input(join(directory,'supercell.d12'),mno_scel_inp)
        
        mno_scel_inp.scf_block.remove('GUESSYMP\n')
        mno_scel_inp.scf_block.insert(-1,'GUESSP\n')
        
        #Displacements
        for k,displ in enumerate(disp_str[1:]):   
            mno_scel = cry_pmg2gui(displ,symmetry=False)
            #mno_scel.atom_number = mno_scel_atomic_numbers

            write_crystal_gui(join(directory,'supercell-00%s.gui'%(k+1)),mno_scel,symm=False)
            write_crystal_input(join(directory,'supercell-00%s.d12'%(k+1)),mno_scel_inp)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



EXITING: a .gui file needs to be specified
Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 2033, in read_cry_gui
    file = open(gui_file, 'r')
FileNotFoundError: [Errno 2] No such file or directory: './isa/mno/MnO.gui'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_15046/1498264602.py", line 1, in <module>
    mno_gui_opt = Crystal_gui().read_cry_gui('./isa/mno/MnO.gui')
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 2038, in read_cry_gui
    sys.exit(1)
SystemExit: 1

During handling of the

TypeError: object of type 'NoneType' has no len()

In [8]:
disp_str[0]

Structure Summary
Lattice
    abc : 13.819301392005695 13.819301392000924 13.819301392000924
 angles : 34.22791099990834 34.227910999988225 34.227910999988225
 volume : 744.5746819421284
      A : 4.695761692232 0.0 12.997034780782
      B : -2.3478808461133998 4.066648915576 12.997034780782
      C : -2.3478808461133998 -4.066648915576 12.997034780782
PeriodicSite: H (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: H (2.3479, 0.0000, 6.4985) [0.5000, 0.0000, 0.0000]
PeriodicSite: H (-1.1739, 2.0333, 6.4985) [0.0000, 0.5000, 0.0000]
PeriodicSite: H (1.1739, 2.0333, 12.9970) [0.5000, 0.5000, 0.0000]
PeriodicSite: H (-1.1739, -2.0333, 6.4985) [0.0000, 0.0000, 0.5000]
PeriodicSite: H (1.1739, -2.0333, 12.9970) [0.5000, 0.0000, 0.5000]
PeriodicSite: H (-2.3479, 0.0000, 12.9970) [0.0000, 0.5000, 0.5000]
PeriodicSite: H (0.0000, 0.0000, 19.4956) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (2.3479, -0.0000, 11.3724) [0.6250, 0.1250, 0.1250]
PeriodicSite: O (0.0000, -0.0000, 4.8739

In [107]:
matrix = SpacegroupAnalyzer(mno_pmg_opt).get_conventional_to_primitive_transformation_matrix()

In [110]:
np.round(np.matmul(np.linalg.inv(matrix),mno_pmg_opt.lattice.matrix),3)

array([[-2.709,  1.564,  0.   ],
       [ 2.709,  1.564,  0.   ],
       [ 0.   ,  0.   , 14.997]])

In [82]:
a = SpacegroupAnalyzer(mno_pmg_opt).get_symmetrized_structure()

In [83]:
a.lattice.matrix

array([[ 1.80606219,  0.        ,  4.99885953],
       [-0.90303109,  1.56409574,  4.99885953],
       [-0.90303109, -1.56409574,  4.99885953]])

In [72]:
Lattice.from_parameters(a,b,c,alpha, beta, gamma )

Lattice
    abc : 5.3151159200021905 5.315115920000354 5.315115920000355
 angles : 34.2279109999083 34.22791099998824 34.22791099998824
 volume : 42.36314758432683
      A : 2.989679431989924 0.0 4.394573259941986
      B : 1.3531190385265182 2.6659429801850294 4.3945732599446385
      C : 0.0 0.0 5.315115920000355

In [75]:
view(AseAtomsAdaptor().get_atoms(mno_pmg))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [411]:
def cart2sph(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    if np.around(az,6) ==  np.around(2*np.pi,6) \
    or np.around(az,6) ==  -np.around(2*np.pi,6):
        az = 0.
    if np.around(az,6) < 0.:
        az = np.round(2*np.pi+az,6)
    return [round(az,6), round(el,6), round(r,6)]

def sph2cart(azelr):
    az = azelr[0]
    el = azelr[1]
    r = azelr[2]
    
    x = r*np.cos(az)*np.sin(el)
    y = r*np.sin(az)*np.sin(el)
    z = r*np.cos(el)
    
    return [x,y,z]

In [428]:
#ONLY ONE
directory = './isa/MnO/test/'

range_scel = int(disp_str[0].num_sites/mno_pmg.num_sites)
for i in range(len(disp_str)):
    for j in range(range_scel):
        disp_str[i].replace(j,1)

spin = []
for i in np.where(np.array(disp_str[0].atomic_numbers)==1)[0]:
    spin.extend(['%s +1'%(i+1)])
for i in np.where(np.array(disp_str[0].atomic_numbers)==25)[0]:
    spin.extend(['%s -1'%(i+1)])

n_spin = len(np.where(np.array(disp_str[0].atomic_numbers)==1)[0])+\
    len(np.where(np.array(disp_str[0].atomic_numbers)==25)[0])
spin = ' '.join(spin)       

mno_scel = cry_pmg2gui(disp_str[0],symmetry=True)
mno_scel.atom_number = mno_scel_atomic_numbers

write_crystal_gui(join(directory,'supercell_symm.gui'),mno_scel)

mno_scel = cry_pmg2gui(disp_str[0],symmetry=False)
mno_scel.atom_number = mno_scel_atomic_numbers

write_crystal_gui(join(directory,'supercell.gui'),mno_scel)


mno_scel_inp = Crystal_input().from_file(join(directory,'MnO.d12'))


#INPUT FILES
mno_scel_inp = Crystal_input().from_file(join(directory,'MnO.d12'))
mno_scel_inp.scf_block.insert(-1,'ATOMSPIN\n')
mno_scel_inp.scf_block.insert(-1,'%s\n'%n_spin)
mno_scel_inp.scf_block.insert(-1,spin+'\n')
mno_scel_inp.scf_block.insert(-1,'SAVEPRED\n')
#mno_scel_inp.scf_block.insert(-1,'MPP\n')
mno_scel_inp.scf_block

write_crystal_input(join(directory,'supercell_symm.d12'),mno_scel_inp)

mno_scel_inp.geom_block.insert(-1,'SYMMREMO\n')
mno_scel_inp.scf_block[-3] = 'GUESSYMP\n'

write_crystal_input(join(directory,'supercell.d12'),mno_scel_inp)

mno_scel_inp.scf_block[-3] = 'GUESSP\n'

#Displacements
for i,disp in enumerate(disp_str[1:]):   
    mno_scel = cry_pmg2gui(disp,symmetry=False)
    mno_scel.atom_number = mno_scel_atomic_numbers

    write_crystal_gui(join(directory,'supercell-00%s.gui'%(i+1)),mno_scel,symm=False)
    write_crystal_input(join(directory,'supercell-00%s.d12'%(i+1)),mno_scel_inp)

## Workflow

## Aurora

In [49]:
for i,func in enumerate(['lda','gga','hybrid']):
    directory = './aurora/phonons/%s'%func
    
    opt_input = Crystal_input().from_file(join(directory,'geom_opt.d12'))
    
    n_disp = 2
    
    shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell_symm.gui'))
    shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell.gui'))

    supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell_symm.gui'))
    supercell_pmg = cry_gui2pmg(supercell_gui)
    supercell_gui = cry_pmg2gui(supercell_pmg)
    write_crystal_gui(join(directory,'supercell_symm.gui'),supercell_gui)

    symm_inp = Crystal_input().from_file('./aurora/phonons/supercell_symm.d12')
    symm_inp.func_block = opt_input.func_block
    write_crystal_input(join(directory,'supercell_symm.d12'),symm_inp)

    no_symm = Crystal_input().from_file('./aurora/phonons/supercell_nosymm.d12')
    no_symm.func_block = opt_input.func_block
    write_crystal_input(join(directory,'supercell.d12'),no_symm)

    disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')
    disp_inp.func_block = opt_input.func_block

    for i in range(1,n_disp+1):

        #disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')

        shutil.copy(join(directory,'supercell-00%s.ext'%i),join(directory,'supercell-00%s.gui'%i))
        #supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell-00%s.gui'%i))
        #supercell_pmg = cry_gui2pmg(supercell_gui)
        #supercell_gui = cry_pmg2gui(supercell_pmg)

        #write_crystal_gui(join(directory,'supercell-00%s.gui'%i),supercell_gui)
        write_crystal_input(join(directory,'supercell-00%s.d12'%i),disp_inp)

In [46]:
directory = './aurora/phonons/test'

opt_input = Crystal_input().from_file(join(directory,'geom_opt.d12'))

n_disp = 2

shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell_symm.gui'))
shutil.copy(join(directory,'supercell.ext'),join(directory,'supercell.gui'))

supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell_symm.gui'))
supercell_pmg = cry_gui2pmg(supercell_gui)
supercell_gui = cry_pmg2gui(supercell_pmg)
write_crystal_gui(join(directory,'supercell_symm.gui'),supercell_gui)

symm_inp = Crystal_input().from_file('./aurora/phonons/supercell_symm.d12')
symm_inp.func_block = opt_input.func_block
write_crystal_input(join(directory,'supercell_symm.d12'),symm_inp)

no_symm = Crystal_input().from_file('./aurora/phonons/supercell_nosymm.d12')
no_symm.func_block = opt_input.func_block
write_crystal_input(join(directory,'supercell.d12'),no_symm)

disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')
disp_inp.func_block = opt_input.func_block

for i in range(1,n_disp+1):

    #disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')

    shutil.copy(join(directory,'supercell-00%s.ext'%i),join(directory,'supercell-00%s.gui'%i))
    #supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell-00%s.gui'%i))
    #supercell_pmg = cry_gui2pmg(supercell_gui)
    #supercell_gui = cry_pmg2gui(supercell_pmg)

    #write_crystal_gui(join(directory,'supercell-00%s.gui'%i),supercell_gui)
    write_crystal_input(join(directory,'supercell-00%s.d12'%i),disp_inp)

In [17]:
directory = './aurora/phonons/lda'
supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell_symm.gui'))
supercell_pmg = cry_gui2pmg(supercell_gui)

In [19]:
a = SpacegroupAnalyzer(supercell_pmg).get_symmetry_operations()

In [43]:
I = np.identity(3,dtype=int)*-1
for op in a:
    if np.sum(np.array(op.rotation_matrix,dtype=int) == I) == 9:
        print('yes')
        


In [10]:
symm_inp.func_block

['a']

In [38]:
directory = './aurora/phonons/lda/'
n_disp = 2

for i in range(1,n_disp+1):
    
    disp_inp = Crystal_input().from_file('./aurora/phonons/supercell_disp.d12')
    
    shutil.copy(join(directory,'supercell-00%s.ext'%i),join(directory,'supercell-00%s.gui'%i))
    supercell_gui = Crystal_gui().read_cry_gui(join(directory,'supercell-00%s.gui'%i))
    supercell_pmg = cry_gui2pmg(supercell_gui)
    supercell_gui = cry_pmg2gui(supercell_pmg)
    
    write_crystal_gui(join(directory,'supercell-00%s.gui'%i),supercell_gui)
    write_crystal_input(join(directory,'supercell-00%s.gui'%i),disp_inp)
    


<class 'crystal_functions.file_readwrite.Crystal_gui'>
<class 'crystal_functions.file_readwrite.Crystal_gui'>


In [51]:
cry_gui = Crystal_gui().read_cry_gui('./mgo.gui')
mgo_pmg = cry_gui2pmg(cry_gui)
mgo_pmg

Structure Summary
Lattice
    abc : 2.9818692962636706 2.9818692962636706 2.9818692962636706
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 18.747821578249994
      A : 0.0 2.1085 2.1085
      B : 2.1085 0.0 2.1085
      C : 2.1085 2.1085 0.0
PeriodicSite: Mg (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: O (0.0000, 0.0000, -2.1085) [-0.5000, -0.5000, 0.5000]

In [52]:
#mgo_pmg.make_supercell(np.identity(3)*2)
mgo_pmg.make_supercell([[-1,1,1],[1,-1,1],[1,1,-1]])
mgo_pmg

Structure Summary
Lattice
    abc : 4.217 4.217 4.217
 angles : 90.0 90.0 90.0
 volume : 74.99128631299997
      A : 4.217 0.0 0.0
      B : 0.0 4.217 0.0
      C : 0.0 0.0 4.217
PeriodicSite: Mg (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Mg (2.1085, 2.1085, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Mg (2.1085, 0.0000, 2.1085) [0.5000, 0.0000, 0.5000]
PeriodicSite: Mg (0.0000, 2.1085, 2.1085) [0.0000, 0.5000, 0.5000]
PeriodicSite: O (0.0000, 0.0000, 2.1085) [0.0000, 0.0000, 0.5000]
PeriodicSite: O (2.1085, 2.1085, 2.1085) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (2.1085, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: O (0.0000, 2.1085, 0.0000) [0.0000, 0.5000, 0.0000]

In [54]:
mgo_pmg.translate_sites(1,[0.,0.,0.003])

In [55]:
len(SpacegroupAnalyzer(mgo_pmg).get_symmetry_operations())
#mgo_gui = cry_pmg2gui(mgo_pmg)

8

In [83]:
write_crystal_gui('./MnO_initial.gui',mgo_gui)

In [88]:
mno_gui = Crystal_gui().read_cry_gui('./MnO_initial.gui')
mno_pmg = cry_gui2pmg(mno_gui)

In [89]:
mno_pmg.replace(2,'H')

In [90]:
mno_pmg.make_supercell(np.identity(3)*2)

In [91]:
mno_pmg

Structure Summary
Lattice
    abc : 10.630231840004381 10.63023184000071 10.63023184000071
 angles : 34.22791099990832 34.22791099998824 34.22791099998824
 volume : 338.9051806746147
      A : 3.61212437864 0.0 9.99771906214
      B : -1.806062189318 3.12819147352 9.99771906214
      C : -1.806062189318 -3.12819147352 9.99771906214
PeriodicSite: Mn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Mn (2.7091, -1.5641, 14.9966) [1.0000, 0.0000, 0.5000]
PeriodicSite: Mn (-0.9030, 1.5641, 4.9989) [0.0000, 0.5000, 0.0000]
PeriodicSite: Mn (1.8061, -0.0000, 19.9954) [1.0000, 0.5000, 0.5000]
PeriodicSite: Mn (1.8061, 0.0000, 4.9989) [0.5000, 0.0000, 0.0000]
PeriodicSite: Mn (0.9030, -1.5641, 9.9977) [0.5000, 0.0000, 0.5000]
PeriodicSite: Mn (0.9030, 1.5641, 9.9977) [0.5000, 0.5000, 0.0000]
PeriodicSite: Mn (0.0000, -0.0000, 14.9966) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (1.8061, 0.0000, 8.7480) [0.6250, 0.1250, 0.1250]
PeriodicSite: O (0.9030, -1.5641, 13.7469) [0.6250, 0.12

In [82]:
for i in np.where(np.array(mno_pmg.atomic_numbers)==25)[0]:
    print(i,' +1')
    #mno_pmg.replace([16, 17, 18, 19, 20, 21, 22, 23],['Mn']*8)

0  +1
1  +1
2  +1
3  +1
4  +1
5  +1
6  +1
7  +1
8  +1
9  +1
10  +1
11  +1
12  +1
13  +1
14  +1
15  +1
24  +1
25  +1
26  +1
27  +1
28  +1
29  +1
30  +1
31  +1
32  +1
33  +1
34  +1
35  +1
36  +1
37  +1
38  +1
39  +1
40  +1
41  +1
42  +1
43  +1
44  +1
45  +1
46  +1
47  +1
48  +1
49  +1
50  +1
51  +1
52  +1
53  +1
54  +1
55  +1
56  +1
57  +1
58  +1
59  +1
60  +1
61  +1
62  +1
63  +1
128  +1
129  +1
130  +1
131  +1
132  +1
133  +1
134  +1
135  +1
136  +1
137  +1
138  +1
139  +1
140  +1
141  +1
142  +1
143  +1
144  +1
145  +1
146  +1
147  +1
148  +1
149  +1
150  +1
151  +1
152  +1
153  +1
154  +1
155  +1
156  +1
157  +1
158  +1
159  +1
160  +1
161  +1
162  +1
163  +1
164  +1
165  +1
166  +1
167  +1
168  +1
169  +1
170  +1
171  +1
172  +1
173  +1
174  +1
175  +1
176  +1
177  +1
178  +1
179  +1
180  +1
181  +1
182  +1
183  +1
184  +1
185  +1
186  +1
187  +1
188  +1
189  +1
190  +1
191  +1


In [94]:
mno_pmg.translate_sites(0,[0.,0.,0.003])

In [74]:
view(AseAtomsAdaptor().get_atoms(cry_gui2pmg(Crystal_gui().read_cry_gui('./isa/MnO_super.out'))))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



EXITING: a .gui file needs to be specified
Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 1957, in read_cry_gui
    file = open(gui_file, 'r')
FileNotFoundError: [Errno 2] No such file or directory: './isa/MnO_super.out.gui'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/f2/8kc7y9697m59bwltxjd42y300000gn/T/ipykernel_62530/1747659824.py", line 1, in <module>
    view(AseAtomsAdaptor().get_atoms(cry_gui2pmg(Crystal_gui().read_cry_gui('./isa/MnO_super.out'))))
  File "/Users/brunocamino/miniconda3/envs/test_env/lib/python3.9/site-packages/crystal_functions/file_readwrite.py", line 1962, in read_cry_gui
    sys.exi

TypeError: object of type 'NoneType' has no len()

In [78]:
view(AseAtomsAdaptor().get_atoms(cry_out2pmg(Crystal_output().read_cry_output('./isa/MnO_super.out'))))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/test_env...>

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [93]:
mno_prim = Structure([[0.0, 2.657557960000818, 2.657557960000818],
                      [2.657557960000818, 0.0, 2.657557960000818],
                      [2.657557960000818, 2.657557960000818, 0.0]],
                     ['Mn','O'],[[0.,0.,0.],[.5,.5,.5]])
mno_conv = SpacegroupAnalyzer(mno_prim).get_conventional_standard_structure()
mno_conv.replace(0,1)
mno_conv.replace(2,1)
mno_prim_symm = SpacegroupAnalyzer(mno_conv).get_primitive_standard_structure()
#a = np.array(mno_prim_symm.equivalent_indices)[:,1:].flatten()
#mno_prim_symm.replace(1,25)
mno_prim_symm.make_supercell(np.identity(3)*2)
write_crystal_gui('mno_super_symm.gui',cry_pmg2gui(mno_prim_symm))

In [70]:
SpacegroupAnalyzer(mno_prim_symm).get_symmetrized_structure()

SymmetrizedStructure
Full Formula (Mn8 H8 O16)
Reduced Formula: MnHO2
Spacegroup: P4/mmm (123)
abc   :   5.963739   5.963739   8.434000
angles:  90.000000  90.000000  90.000000
Sites (32)
  #  SP       a     b     c  Wyckoff
---  ----  ----  ----  ----  ---------
  0  Mn    0.25  0.25  0.25  8d
  1  H     0     0     0     8a
  2  O     0.25  0.25  0     8c
  3  O     0     0     0.25  8b

In [25]:
cry_bands = Properties_input().make_bands_block()

TypeError: make_bands_block() missing 4 required positional arguments: 'k_path', 'n_kpoints', 'first_band', and 'last_band'

In [26]:
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from crystal_functions.file_readwrite import write_properties_input
from crystal_functions.file_readwrite import Crystal_output
from crystal_functions.file_readwrite import Properties_input
from crystal_functions.convert import cry_out2pmg


#Create the bands input object
bands_input = Properties_input()
#Add the newk block to the input object
bands_input.make_newk_block(12,24)

#Read the structure
mgo = Crystal_output().read_cry_output('aurora/GaP.out')
mgo = cry_out2pmg(mgo)
mgo_prim = SpacegroupAnalyzer(mgo).get_primitive_standard_structure(international_monoclinic=False)

#Obtain the k path object
k_path = HighSymmKpath(mgo_prim)

n_kpoints = 200
first_band = 1
last_band = 26
bands_input.make_bands_block(k_path,n_kpoints,first_band,last_band)

#Write the input
write_properties_input('aurora/GaP.d3',bands_input)

In [73]:
view(cry_gui2ase(Crystal_gui().read_cry_gui('../anna/second_structure/li110-li2s110_sym.gui')))

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/qc/bin/p...>

In [41]:
#mno_pmg_opt.replace(0,6)
from ase.optimize import BFGS
atoms = AseAtomsAdaptor().get_atoms(mno_pmg_opt)
from ase.calculators.lj import LennardJones

atoms.calc = LennardJones()
atoms.get_potential_energy()
dyn = BFGS(atoms)
dyn.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 09:22:12       -0.358112        0.0000


True

In [32]:
from ase import Atoms
from ase.optimize import BFGS
from ase.calculators.emt import EMT
import numpy as np
d = 0.9575
t = np.pi / 180 * 104.51
water = Atoms('H2O',
              positions=[(d, 0, 0),
                         (d * np.cos(t), d * np.sin(t), 0),
                         (0, 0, 0)],
              calculator=EMT())
dyn = BFGS(water)
dyn.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 09:17:35        2.769632        8.6091
BFGS:    1 09:17:35        1.930507        1.6415
BFGS:    2 09:17:35        1.884227        0.5354
BFGS:    3 09:17:35        1.879314        0.0496


True